# Synthetic Data Generation for Skills Tutorial using LLaMA

**Key Update:**
The biggest change in this improved workflow is that we use the Llama model to directly classify samples into skill routes using guided choices. This replaces the earlier approach where LoRA adapters were used with Mixtral for routing/classification. This new method leverages Llama's strong multi-class classification ability and simplifies the workflow by removing the need for adapter-based routing.

This tutorial demonstrates how to use the SDG repository to generate synthetic question-answer pairs from documents using large language models like LLaMA 3.3 70B. We will also generate data using the Mixtral model for comparison. We'll cover:

1. Setting up the environment
2. Connecting to LLM servers
3. Configuring the data generation pipeline
4. Generating data


In [ ]:
# Enable auto-reloading of modules - useful during development
%load_ext autoreload
%autoreload 2

### Setup Instructions

Before running this notebook, you'll need to:

```bash 
pip install sdg-hub==0.1.0a4

In [ ]:
# Import required libraries
# datasets: For handling our data
# OpenAI: For interfacing with the LLM servers
# SDG components: For building our data generation pipeline
from datasets import load_dataset, Dataset
from openai import OpenAI

from sdg_hub.flow import Flow
from sdg_hub.pipeline import Pipeline
from sdg_hub.sdg import SDG
from sdg_hub.registry import PromptRegistry

### Setting up LLaMA 3.3 70B Model

First, we need to host the LLaMA model using vLLM. This creates an OpenAI-compatible API endpoint.

1. Start the vLLM server (run in terminal):
```bash
CUDA_VISIBLE_DEVICES=0,1,2,3,4,5,6,7 python -m vllm.entrypoints.openai.api_server \
    --model meta-llama/Llama-3.3-70B-Instruct \
    --dtype float16 \
    --tensor-parallel-size 8 
```

2. Connect to the model using OpenAI client below:

In [ ]:
# Configure OpenAI client to connect to our local vLLM server
endpoint = f"http://localhost:8000/v1"
openai_api_key = "EMPTY"  # vLLM doesn't require real API key
openai_api_base = endpoint

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# Verify we can see the model
teacher_model = client.models.list().data[0].id
print(f"Connected to model: {teacher_model}")

### Configure and Chain Data Generation Pipelines
In this section, we'll demonstrate how to chain two Synthetic Data Generation (SDG) pipelines using two different flow YAML configurations. This is useful when you want to perform multi-stage data processing, such as generating initial data with one pipeline and then refining, critiquing, or further processing it with a second pipeline.
#### Steps:
1. **Load SDG Flow configurations from YAML:**
* `synth-skills-llama3.3.yaml` for the first stage (e.g., initial data generation)
* `agentic-skills-llama3.3.yaml` for the second stage (e.g., skill routing, critique, revision etc)
2. **Initialize the SDG pipeline with both flows:**
* Pass both flow configurations as a list of Pipeline objects to a single SDG instance.
* Set processing parameters such as batch_size, num_workers, and save_freq.
3. **Run the chained pipeline:**
The SDG pipeline will automatically process your dataset through each stage in sequence, passing the output of one pipeline as the input to the next.

This approach allows for modular, flexible, and reusable data generation workflows, where each pipeline can focus on a specific stage of the process.


In [4]:
# Load the flow configuration from YAML file
flow_cfg1 = Flow(client).get_flow_from_file("synth-skills-llama3.3.yaml")
flow_cfg2 = Flow(client).get_flow_from_file("agentic-skills-llama3.3.yaml")

# Initialize the SDG pipeline with processing parameters
pipeline = SDG(
    [Pipeline(flow_cfg1), Pipeline(flow_cfg2)],
    num_workers=1,      # Number of parallel workers
    batch_size=1,       # Batch size for processing
    save_freq=1000,     # How often to save checkpoints
)

### Load and Prepare Seed Data
We will import the skills sample data set to generate question-answer pairs. 

In [5]:
# Load the seed data from JSON file
seed_data_path = "../instructlab/skills/sample_data/unstructured_to_mdtable_seeds.jsonl"  # You can replace with your data path here.
ds = load_dataset('json', data_files=seed_data_path, split='train')

In [6]:
# For testing, we'll use just two examples
ds = ds.select(range(2))

### Generate Data with LLaMA 3.3

Now we'll use our configured pipeline to generate synthetic question-answer pairs.

In [ ]:
# Generate synthetic data and save checkpoints
generated_data = pipeline.generate(ds, checkpoint_dir="Tmp")

#### Examples:

Each example below shows the generated sample content along with the route assigned by the Llama 3.3 model, illustrating how the model classifies different types of input data.

In [12]:
generated_data[0]

{'task_description': 'Convert the following unstructured user feedback into a structured markdown table.',
 'seed_question': "Been using the new dashboard for a few days. It's way faster than the previous one, really appreciate the snappy filters. But export to CSV seems broken — nothing happens when I click it. Also, dark mode resets every time I log in.\n\nI would like to convert the above feedback into a markdown table with columns for Feature, Feedback and Sentiment.",
 'seed_response': "| Feature           | Feedback                                                           | Sentiment |\n|------------------|--------------------------------------------------------------------|-----------|\n| Dashboard        | Much faster than previous version, filters are responsive.         | Positive  |\n| Export to CSV    | Clicking the export button doesn't trigger a download.             | Negative  |\n| Dark Mode        | Resets to light mode on login.                                     | 

In [13]:
generated_data[1]

{'task_description': 'Convert the following unstructured user feedback into a structured markdown table.',
 'seed_question': "Been using the new dashboard for a few days. It's way faster than the previous one, really appreciate the snappy filters. But export to CSV seems broken — nothing happens when I click it. Also, dark mode resets every time I log in.\n\nI would like to convert the above feedback into a markdown table with columns for Feature, Feedback and Sentiment.",
 'seed_response': "| Feature           | Feedback                                                           | Sentiment |\n|------------------|--------------------------------------------------------------------|-----------|\n| Dashboard        | Much faster than previous version, filters are responsive.         | Positive  |\n| Export to CSV    | Clicking the export button doesn't trigger a download.             | Negative  |\n| Dark Mode        | Resets to light mode on login.                                     | 